Jetsim : Road Following
===

### Step 5 from README.md

---

Right click and open 'this' notebook **JESTIM_interactive_regression.ipyn** in Google colab.

Open 'Edit/Notebook settings' toolbar in Google Colab and select **GPU**

If you get any runtime errors at anytime, go to 'Runtime' toolbar and click 'Restart Runtime'. Rerun all cells.

---

---

### Check DIY Robocar virtual track server is [ONLINE](https://www.twitch.tv/roboticists)
You should see an images of the warren track maybe with or without cars. 

---


In [ ]:
#initialize google colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
print("DONE")


'jestim-google-colab' dirctory
===

In [ ]:
cd gdrive/MyDrive/jetsim_googlecolab 

Git clone gym-donkeycar to Google Drive or Update
===

#### If you have not installed before, run the cell below

In [ ]:
!git clone https://github.com/tawnkramer/gym-donkeycar
%cd gym-donkeycar/
!python3 -m pip install -e .[gym-donkeycar] --user
%cd ..
%mv component.py gym-donkeycar/
%mv gyminterface.py gym-donkeycar/

#### If you have already installed, then run the cell below to upgrade

In [ ]:
#UPGRADE gym-donkeycar ONLY#
!pip install git+https://github.com/tawnkramer/gym-donkeycar --upgrade

### IMPORTANT!!!

- Go to Google Drive and check if gym-donkeycar folder is installed. 
- gyminterface.py and component.py should be in gym-donkeycar folder.

Connect to Donkeycar simulator: (Remote only)
===

NOTE: This notebook is running locally on a Google GPU so it is not local to your network. That is why you must use 'remote' server settings to see your model perform in a simulator. 

## FOR COSMOS, USE METHOD 1

### Method 1 : To DIY Robocar server (provided) / someone elses computer.
- Use DIY robocar server or another provided IP address and copy to remote settings in cell below.

### Method 2 : To your own host computer.
- Configure your router settings port forwarding to your host computer IP address with port 9091 to 9091.
- Google search 'my ip address' and copy to remote settings in cell below.

Create the racecar dictionary
===

---

Customize your car settings.

Select track to race on. Warren is the default.

Enter the method 1 or 2 IP address as 'remote' for remote connection.

---

In [ ]:
# We are initilizaing the configuration file (Similar to myconfig.py)

GYM_DICT={
  'car':{
    'racer_name': 'Jetsim_NAME', #CHANGE
    'bio' : 'Triton_AI_Jetsim = Jetracer', #CHANGE
    'country' : 'US',
    "guid": "GO_JETSIM",
    'body_style' : 'car01',
    'body_rgb': (24, 43, 200),      #(R, G, B)
    'car_name' : 'Training_Jetsim', #CHANGE
    'font_size' : 50,
    #camera settings
    "fov" : 80,
    "fish_eye_x" : 0.0, 
    "fish_eye_y" : 0.0, 
    "img_w" : 224, 
    "img_h" : 224, 
    "img_d" : 3, 
    "img_enc" : 'JPG', 
    "offset_x" : 0.0, #sides
    "offset_y" : 2.0, #height #Jetsim 2.0
    "offset_z" : 0.0, #forward
    "rot_x" : 20.0,   #tilt #Jetsim 0.0
    #"rot_y": 180,    #rotate
    },

    #connection type
    # 'default_connection': 'local',
    'default_connection': 'remote',

    'local_connection':{
      'scene_name': 'warren', # warren | thunderhill | mini_monaco | roboracingleague_1 | generated_track | generated_road | warehouse | sparkfun_avc | waveshare

######YOUR HOST COMPUTER IP ADDRESS######
      'host': '127.0.0.1', #UPDATE WITH COMPUTER IP ADDRESS HOSTING SIM
      'port': 9091,
      'artificial_latency': 0},

    'remote_connection':{
        'scene_name': 'warren',
###### REMOTE IP ADDRESS ######
        'host': 'donkey-sim.roboticist.dev', # DIY Robocar virtual track server = 'donkey-sim.roboticist.dev' 
        'port': 9091,
        'artificial_latency': 0},# Ping the remote simulator. millisecond here.
        
    'lidar':{
        'enabled': False,
        'deg_inc': 1, # Degree increment between each ray of the lidar
        'max_range': 30.0}, # Max range of the lidar laser
    }


Send dictionary to Triton Racer Gym interface
===

### Make sure the **DIY Robocar server or Donkeycar Simulator** is ONLINE before running the code below.


In [ ]:
import time

%cd gym-donkeycar/
from gyminterface import GymInterface
gym = GymInterface(gym_config=GYM_DICT)
time.sleep(1)
imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False)
time.sleep(1)
print(imageSeen.shape)
%cd ..

#If you run this cell more than once it will generate more cars in the sim. Turn simulator application off and back on to delete all cars. 

---

This will initialize the **racecar in the simulator**. Check if the car **populated** in the sim.

---


In [ ]:
#If the custom car setting did not load correctly, uncomment this to take car out of sim and then re-run the cell above.

gym.onShutdown()

Import and load the model. Input your model name.
==

In [ ]:
import torch
import torchvision

MODEL_NAME = 'NAME.pth'
CATEGORIES = ['m2_images']
device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

# Make sure the model you select is in the 'models' folder within this Google drive setup.
model.load_state_dict(torch.load('./models/' + MODEL_NAME))
print("DONE")

Run the model
===

In [ ]:
from utils import preprocess
import numpy as np
import time
import datetime
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output


#****************PRIMARY CONTROLS*****************
STEERING_GAIN = 1.0             #Scalar or intesity of steering response, usally matches THROTTLE_GAIN.
THROTTLE_GAIN = 1.0             #Scalar or intesity of throttle, usally matches STEERING_GAIN.
MAX           = 0.9             #Limiter for controlling speed.
#********************OPTIONS**********************
AUTO_RESET    = True            #reset the car to the start line if crash detected.
BOOST         = False           #Inital booster THROTTLE = 1.0 at start of race for 5 seconds.
LIMITER       = False           # THROTTLE = MAX 
PRINT         = False           #outputs steering & throttle values
STRAIGHT_AWAY = False           #Straight away for max speed based on x and z coordinate from step function.
STRAIGHT_AWAY_THROTTLE = 1.0    #Straight away max speed.
#*******************INITIAL***********************
STEERING_BIAS = 0.0             #Steering offset. 
THROTTLE_BIAS = 0.0             #Throttle offset, creeps forward at minimum speed.
BREAKS        = 0.0             #Final break value.
t0 = datetime.datetime.now()    #for lap time
out = 0                         #counter for clearing steering & throttle outputs
speed_last = 0                  #auto reset
mark = 0                        #auto reset
stuck = datetime.datetime.now() #auto reset
#*************************************************



#Main Jetsim model loop
print('JETSIM ACTIVATED')
THROTTLE = 1.0
STEERING = 0.0
try: 
    while True:
        
        #_________________________Boost_______________________
        #Inital booster THROTTLE = 1.0 at start of race for 5 seconds.
        if BOOST:
            image, x, y, z, speed, cte, lidar = gym.step(0, 1.0, 0.0, False);
            print("BOOST")
            time.sleep(5) #boost for 5 seconds
            print("DRIVING")
            BOOST = False
        #_____________________________________________________



        #___________________Image processing__________________
        #Send controls to gym wrapper and returns image, coordinates, etc.
        image, x, y, z, speed, cte, lidar = gym.step(STEERING, THROTTLE, BREAKS, False);
        image = preprocess(image).half()
        output = model(image).detach().cpu().numpy().flatten()
        S = float(output[0]) #-1.0 to 1.0 steering decision.
        T = float(output[1]) ##-1.0 to 1.0 throttle decision.
        #_____________________________________________________



        #_____________Steering & Throttle control_____________
        #optimize steering and throttle value. Set the limit on the top speed. 
        STEERING = round(S * STEERING_GAIN + STEERING_BIAS, 2)  
        THROTTLE = round(T * THROTTLE_GAIN + THROTTLE_BIAS, 2)
        if LIMITER: #Limiter
            if THROTTLE > MAX: #maximum speed
                THROTTLE = MAX
        #____________________________________________________



        #____________________Straight away_____________________
        #if car is on the straightest path of the track, go max speed. Warren track only because of coordinates.
        if STRAIGHT_AWAY and x <= 45 and x >= 5 and z <= -20 and z >= -30: #coordinates
            THROTTLE = STRAIGHT_AWAY_THROTTLE
            STEERING = STEERING*0.5 #reduce oversteer
        #____________________________________________________



        #____________________Print value_____________________
        #print steering and throttle value to see realtime control values.
        if PRINT: #output value
            print("Steering = "+str(STEERING)+" Throttle  = "+str(THROTTLE))
            if out == 10:
                clear_output()
                out = 0
            out+=1
        #____________________________________________________

            

        #____________________Auto reset______________________
        #reset the car to the start line if crash detected.
        if AUTO_RESET and round(speed,0) == round(speed_last,0):
          mark = datetime.datetime.now()
          check = mark - stuck
          if (str(check) > '0:00:07.000000'):
            print('NOT MOVING')
            stuck = datetime.datetime.now()
            BOOST = True
            gym.step(0.0, 0.0, 100.0, True)
            time.sleep(1)
        else: 
          speed_last = speed
          mark = stuck
          stuck = datetime.datetime.now()
        #____________________________________________________



        #___________________Show lap time___________________
        #displays by default. Cannot show steering and throttle output at the same time. 
        if x>=29 and x<=30 and z <= -20 and z >= -30:
            time_print = datetime.datetime.now() - t0
            if (str(time_print) > '0:00:01.000000'):
                print (time_print)
            t0 = datetime.datetime.now()
        #____________________________________________________


#####STOP the main loop select "Kernel" in the drop down menu and click "Interrupt Kernel"#####
except KeyboardInterrupt:
    
    #Resets steering and throttle to 0, applies breaks and resets for 1 second.
    print("!!!STOP!!!")
    gym.step(0.0, 0.0, 100.0, True)
    time.sleep(1)

In [ ]:
#RESET run this cell multiple times to stop the car if it still running.
gym.step(0.0, 0.0, 100.0, True)

If finished - YOU MUST Take car out of SERVER!!!
===

In [ ]:
gym.onShutdown()